In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
data = pd.read_feather('tavbase/Sample.feather')

In [3]:
def deep_learning_dnn(df_dl, dep_var, classes):
    # Separa a variável dependente das demais
    deep_feat = df_dl.drop(columns=[dep_var], axis=1)
    deep_label = df_dl[dep_var]
    # Verifica os tipos das variáveis
    categorical_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) == 2 or deep_feat[col].dtype == 'O']
    continuous_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) > 2 and (deep_feat[col].dtype == 'int64' or deep_feat[col].dtype == 'float64')]
    # Verifica as colunas para normalização - as demais serão discretizadas - Função Bucketize do Tensor Flow
    cols_to_scale = continuous_columns[:]
    #cols_to_scale.remove('meses')
    # Ajusta as bases de treino e de teste
    XX_T = df_dl.drop(columns=[dep_var], axis=1)
    XX_t = df_dl.drop(columns=[dep_var], axis=1)
    yy_T = df_dl[dep_var]
    yy_t = df_dl[dep_var]
    # Normaliza as variáveis nas bases de treino e teste
    scaler = StandardScaler()
    XX_T.loc[:, cols_to_scale] = scaler.fit_transform(XX_T.loc[:, cols_to_scale]) ## Treinamento
    XX_t.loc[:, cols_to_scale] = scaler.fit_transform(XX_t.loc[:, cols_to_scale]) ## Teste
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_object_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_hash_bucket(key=col, hash_bucket_size=1000), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df_dl[col].dtype == 'O']
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_integer_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_identity(key=col, num_buckets=2), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df[col].dtype=='int64']
    continuous_feat_cols = [tf.feature_column.numeric_column(key=col) for col in continuous_columns] 
    feat_cols = categorical_object_feat_cols + \
                categorical_integer_feat_cols + \
                continuous_feat_cols
    # Rotina de DNN (Deep Neural Network)
    input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_T, yy_T, batch_size=50, num_epochs=1000, shuffle=True)
    pred_input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_t, batch_size=50, shuffle=False)
    DNN_model = tf.estimator.DNNClassifier(hidden_units=[10, 10, 10], feature_columns=feat_cols, n_classes=classes)
    DNN_model.train(input_fn=input_fun, steps=5000)
    # Resgata os resultados da DNN
    predictions = DNN_model.predict(pred_input_fun)
    pred = list(predictions)
    return pred

In [4]:
## Aula 14 - Probabilidade
dimensao = 'State'
medidas = ['Sales', 'Quantity', 'Profit']
grupo = data.groupby(dimensao)[medidas].mean().reset_index()
grupo['Benefit'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)
grupo = grupo.set_index(dimensao)

In [5]:
grupo

,Sales,Quantity,Profit,Benefit
State,,,,
Alabama,319.846557,4.196721,94.865989,1
Arizona,157.508933,3.848214,-15.303235,0
Arkansas,194.635500,4.000000,66.811452,1
California,228.729451,3.831584,38.171608,1
Colorado,176.418231,3.807692,-35.867351,0
Connecticut,163.223866,3.426829,42.823071,1
Delaware,285.948635,3.822917,103.930988,1
District of Columbia,286.502000,4.000000,105.958930,1
Florida,233.612815,3.600522,-8.875461,0


In [6]:
probabilidade = deep_learning_dnn(grupo, 'Benefit', 2)
probabilidade_classe = []
for i in range(len(probabilidade)):
    probabilidade_classe.append(probabilidade[i]["class_ids"][0])
probabilidade_prob0 = []
for i in range(len(probabilidade)):
    probabilidade_prob0.append(probabilidade[i]["probabilities"][0])
probabilidade_prob1 = []
for i in range(len(probabilidade)):
    probabilidade_prob1.append(probabilidade[i]["probabilities"][1]) 
grupo['dl_classe'] = probabilidade_classe
grupo['lucro_0'] = probabilidade_prob0
grupo['lucro_1'] = probabilidade_prob1
grupo = grupo.reset_index()
grupo



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\duduf\\AppData\\Local\\Temp\\tmpxcrt8g0i', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read

,State,Sales,Quantity,Profit,Benefit,dl_classe,lucro_0,lucro_1
0,Alabama,319.846557,4.196721,94.865989,1,1,0.455106,0.544894
1,Arizona,157.508933,3.848214,-15.303235,0,1,0.446741,0.553259
2,Arkansas,194.635500,4.000000,66.811452,1,1,0.463476,0.536524
3,California,228.729451,3.831584,38.171608,1,1,0.462329,0.537671
4,Colorado,176.418231,3.807692,-35.867351,0,1,0.443441,0.556559
5,Connecticut,163.223866,3.426829,42.823071,1,1,0.454293,0.545707
6,Delaware,285.948635,3.822917,103.930988,1,1,0.460378,0.539622
7,District of Columbia,286.502000,4.000000,105.958930,1,1,0.455671,0.544329
8,Florida,233.612815,3.600522,-8.875461,0,1,0.449435,0.550565
9,Georgia,266.825217,3.831522,88.315453,1,1,0.456969,0.543031


In [7]:
grupo.to_feather('tavbase/probabilidade_estado.feather')